# Cleaning Demographic Data

In [ ]:
import pandas as pd

In [2]:
# Loading the files you just saved
df_demo = pd.read_csv('Raw Datasets/raw_demographic.csv')
df_bio = pd.read_csv('Raw Datasets/raw_biometric.csv')

In [3]:
df_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   date           object
 1   state          object
 2   district       object
 3   pincode        int64 
 4   demo_age_5_17  int64 
 5   demo_age_17_   int64 
dtypes: int64(3), object(3)
memory usage: 94.8+ MB


In [4]:
df_demo['date'] = pd.to_datetime(df_demo['date'])
df_bio['date'] = pd.to_datetime(df_bio['date'])

df_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2071700 entries, 0 to 2071699
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   date           datetime64[ns]
 1   state          object        
 2   district       object        
 3   pincode        int64         
 4   demo_age_5_17  int64         
 5   demo_age_17_   int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 94.8+ MB


In [5]:
# 1. Create the total update column
df_demo['total_updates'] = df_demo['demo_age_5_17'] + df_demo['demo_age_17_']

# 2. Group by state and sum the updates, then sort descending
state_updates = df_demo.groupby('state')['total_updates'].sum().sort_values(ascending=False)

# 3. Print the result
print("--- Demographic Updates per State (Check for Typos) ---")
print(state_updates)

--- Demographic Updates per State (Check for Typos) ---
state
Uttar Pradesh           8542328
Maharashtra             5054602
Bihar                   4814350
West Bengal             3872172
Madhya Pradesh          2912938
                         ...   
BALANAGAR                     2
100000                        2
Raja Annamalai Puram          1
Puttenahalli                  1
Nagpur                        1
Name: total_updates, Length: 65, dtype: int64


In [6]:


# Group and sort
state_updates = df_demo.groupby('state')['total_updates'].sum().sort_values(ascending=False)

# Force the print to show ALL 65 rows
print("--- FULL STATE AUDIT (All 65 entries) ---")
print(state_updates.to_string())

--- FULL STATE AUDIT (All 65 entries) ---
state
Uttar Pradesh                               8542328
Maharashtra                                 5054602
Bihar                                       4814350
West Bengal                                 3872172
Madhya Pradesh                              2912938
Rajasthan                                   2817615
Andhra Pradesh                              2295505
Tamil Nadu                                  2212228
Chhattisgarh                                2005434
Gujarat                                     1824327
Karnataka                                   1695285
Telangana                                   1629908
Delhi                                       1438934
Jharkhand                                   1401189
Haryana                                     1166140
Odisha                                      1083237
Assam                                       1012578
Punjab                                       881895
Kerala          

In [7]:
!pip install fuzzywuzzy python-Levenshtein
from fuzzywuzzy import process

# 1. Get the list of unique states from your messy df_demo
unique_states = df_demo['state'].unique().tolist()

# 2. Set a similarity threshold (80-90 is usually the sweet spot)
threshold = 85
potential_duplicates = []

print(f"Scanning {len(unique_states)} unique names for 'shit'...")

# 3. Compare each name against the others
for state in unique_states:
    # Find matches that aren't the exact same word
    matches = process.extract(state, [s for s in unique_states if s != state], limit=2)
    
    for match, score in matches:
        if score >= threshold:
            # Sort to avoid [A, B] and [B, A] duplicates in our results
            pair = sorted([state, match])
            if pair not in potential_duplicates:
                potential_duplicates.append(pair)

# 4. Display the results
print("\n--- Potential State Duplicates Found ---")
for pair in potential_duplicates:
    print(f"Matches: {pair[0]} <---> {pair[1]}")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 27.2 MB/s  0:00:00

   ---------- ----------------------------- 1/4 [rapidfuzz]
   -------------------- ------------------- 2/4 [Levenshtein]
   ---------------------------------------- 4/4 [python-Levenshtein]

Scanning 65 unique names for 'shit'...

--- Potential State Duplicates Found ---
Matches: Andhra Pradesh <---> andhra pradesh
Matches: Andhra Pradesh <---> Madhya Pradesh
Matches: West Bengal <---> west Bengal
Matches: WEST BENGAL <---> West Bengal
Matches: Odisha <---> odisha
Matches: ODISHA <---> Odisha
Matches: Madhya Pradesh <---> andhra pradesh
Matches: Chhatisgarh <---> Chhattisgarh
Matches: Jammu & Kashmir <---> Jammu and Kashmir
Matches: Dadra and Nagar Haveli and Daman and Diu <---> Jammu and Kashmir
Matches: Daman & Diu <---> Daman and Diu
Matches: Dadra and Nagar Haveli and Daman and Diu <---> Daman and Diu
Matches: Dadra and Nagar Haveli <---> 

In [8]:
# 1. First, basic standardizing to kill easy duplicates
df_demo['state'] = df_demo['state'].str.strip()

# 2. The Comprehensive Clean Map (Filtered from your fuzzy results + Manual Audit)
demo_state_map = {
    # West Bengal cleanup
    'West Bangal': 'West Bengal', 'West  Bengal': 'West Bengal', 
    'Westbengal': 'West Bengal', 'WEST BENGAL': 'West Bengal', 
    'West bengal': 'West Bengal', 'WESTBENGAL': 'West Bengal', 
    'west Bengal': 'West Bengal', 'West Bengli': 'West Bengal',
    
    # Other Typos & Case
    'andhra pradesh': 'Andhra Pradesh', 'ODISHA': 'Odisha', 
    'odisha': 'Odisha', 'Orissa': 'Odisha', 'Chhatisgarh': 'Chhattisgarh',
    'Pondicherry': 'Puducherry', 'Puducherry': 'Puducherry', 
    'Uttaranchal': 'Uttarakhand',
    
    # Union Territory Mergers
    'Andaman & Nicobar Islands': 'Andaman and Nicobar Islands',
    'Daman and Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Daman & Diu': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra and Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Dadra & Nagar Haveli': 'Dadra and Nagar Haveli and Daman and Diu',
    'Jammu & Kashmir': 'Jammu and Kashmir',
    
    # City-to-State Mapping (Moving cities back to their parent states)
    'Nagpur': 'Maharashtra',
    'Darbhanga': 'Bihar',
    'Jaipur': 'Rajasthan',
    'Madanapalle': 'Andhra Pradesh',
    'BALANAGAR': 'Telangana',
    'Puttenahalli': 'Karnataka',
    'Raja Annamalai Puram': 'Tamil Nadu'
}

# 3. Apply the Map
df_demo['state'] = df_demo['state'].replace(demo_state_map)

# 4. Filter out the numeric '100000' junk
df_demo = df_demo[df_demo['state'] != '100000'].copy()

print("States are now cleaned and unified!")

States are now cleaned and unified!


In [9]:
# 1. Get the list of unique states from your messy df_demo
unique_states = df_demo['state'].unique().tolist()

# 2. Set a similarity threshold (80-90 is usually the sweet spot)
threshold = 85
potential_duplicates = []

print(f"Scanning {len(unique_states)} unique names ")

# 3. Compare each name against the others
for state in unique_states:
    # Find matches that aren't the exact same word
    matches = process.extract(state, [s for s in unique_states if s != state], limit=2)
    
    for match, score in matches:
        if score >= threshold:
            # Sort to avoid [A, B] and [B, A] duplicates in our results
            pair = sorted([state, match])
            if pair not in potential_duplicates:
                potential_duplicates.append(pair)

# 4. Display the results
print("\n--- Potential State Duplicates Found ---")
for pair in potential_duplicates:
    print(f"Matches: {pair[0]} <---> {pair[1]}")

Scanning 36 unique names 

--- Potential State Duplicates Found ---
Matches: Andhra Pradesh <---> Madhya Pradesh
Matches: Dadra and Nagar Haveli and Daman and Diu <---> Jammu and Kashmir
Matches: Andaman and Nicobar Islands <---> Jammu and Kashmir


In [10]:
# Group and sort
state_updates = df_demo.groupby('state')['total_updates'].sum().sort_values(ascending=False)

# Force the print to show ALL 65 rows
print("--- FULL STATE AUDIT (All 65 entries) ---")
print(state_updates.to_string())

--- FULL STATE AUDIT (All 65 entries) ---
state
Uttar Pradesh                               8542328
Maharashtra                                 5054603
Bihar                                       4814352
West Bengal                                 3872737
Madhya Pradesh                              2912938
Rajasthan                                   2817617
Andhra Pradesh                              2295584
Tamil Nadu                                  2212229
Chhattisgarh                                2005438
Gujarat                                     1824327
Karnataka                                   1695286
Telangana                                   1629910
Delhi                                       1438934
Jharkhand                                   1401189
Haryana                                     1166140
Odisha                                      1112065
Assam                                       1012578
Punjab                                       881895
Kerala          

In [11]:
df_demo.head(20)

,date,state,district,pincode,demo_age_5_17,demo_age_17_,total_updates
0,2025-03-01,Uttar Pradesh,Gorakhpur,273213,49,529,578
1,2025-03-01,Andhra Pradesh,Chittoor,517132,22,375,397
2,2025-03-01,Gujarat,Rajkot,360006,65,765,830
3,2025-03-01,Andhra Pradesh,Srikakulam,532484,24,314,338
4,2025-03-01,Rajasthan,Udaipur,313801,45,785,830
5,2025-03-01,Rajasthan,Sikar,332028,28,285,313
6,2025-03-01,Karnataka,Tumakuru,572201,88,332,420
7,2025-03-01,Uttar Pradesh,Gorakhpur,273211,61,836,897
8,2025-03-01,Andhra Pradesh,Kurnool,518313,83,986,1069
9,2025-03-01,West Bengal,Paschim Medinipur,721148,13,281,294


In [12]:
# 1. Fix the State-as-City rows and ensure District is correct
city_to_state_dist = {
    'Nagpur': ('Maharashtra', 'Nagpur'),
    'Darbhanga': ('Bihar', 'Darbhanga'),
    'Jaipur': ('Rajasthan', 'Jaipur'),
    'Madanapalle': ('Andhra Pradesh', 'Annamayya'), # Madanapalle is in Annamayya  district
    'BALANAGAR': ('Telangana', 'Medchal'),     # Balanagar is usually under Rangareddy
    'Puttenahalli': ('Karnataka', 'Bengaluru Urban'),
    'Raja Annamalai Puram': ('Tamil Nadu', 'Chennai')
}

for city, (correct_state, correct_dist) in city_to_state_dist.items():
    # Find rows where the state was accidentally the city name
    mask = df_demo['state'] == city
    df_demo.loc[mask, 'district'] = correct_dist
    df_demo.loc[mask, 'state'] = correct_state

print("City-to-State and District alignment complete.")

City-to-State and District alignment complete.


In [13]:
df_demo.tail(20)

,date,state,district,pincode,demo_age_5_17,demo_age_17_,total_updates
2071680,2025-10-31,Uttar Pradesh,Bulandshahr,203395,0,2,2
2071681,2025-10-31,Uttar Pradesh,Bulandshahr,203397,0,3,3
2071682,2025-10-31,Uttar Pradesh,Bulandshahr,245401,0,21,21
2071683,2025-10-31,Uttar Pradesh,Bulandshahr,245405,3,44,47
2071684,2025-10-31,Uttar Pradesh,Chandauli,232107,0,10,10
2071685,2025-10-31,Uttar Pradesh,Chandauli,232110,4,74,78
2071686,2025-10-31,Uttar Pradesh,Chitrakoot,210204,0,24,24
2071687,2025-10-31,Uttar Pradesh,Chitrakoot,210208,7,43,50
2071688,2025-10-31,Uttar Pradesh,Deoria,274182,4,23,27
2071689,2025-10-31,Uttar Pradesh,Deoria,274202,2,45,47


In [14]:
from fuzzywuzzy import process

# 1. Get list of unique states
states = df_demo['state'].unique()

print("Searching for District Duplicates (State-by-State)...")
print("-" * 50)

for state in states:
    # Filter districts for the current state
    districts = df_demo[df_demo['state'] == state]['district'].unique().tolist()
    
    # We need at least 2 districts to compare
    if len(districts) < 2:
        continue
        
    found_for_state = False
    for dist in districts:
        # Find matches within the same state
        matches = process.extract(dist, [d for d in districts if d != dist], limit=1)
        
        for match, score in matches:
            if score >= 85:
                # To avoid printing (A, B) and (B, A), we use a sorted check
                print(f"In {state}: Potential duplicates {sorted([dist, match])} (Score: {score})")
                found_for_state = True
                # Remove from list to avoid reverse-pair printing
                districts.remove(match) 

print("-" * 50)
print("Search complete.")

Searching for District Duplicates (State-by-State)...
--------------------------------------------------
In Uttar Pradesh: Potential duplicates ['Bara Banki', 'Barabanki'] (Score: 95)
In Uttar Pradesh: Potential duplicates ['Ambedkar Nagar', 'Sant Ravidas Nagar Bhadohi'] (Score: 86)
In Uttar Pradesh: Potential duplicates ['Agra', 'Prayagraj'] (Score: 90)
In Uttar Pradesh: Potential duplicates ['Balrampur', 'Rampur'] (Score: 90)
In Uttar Pradesh: Potential duplicates ['Rae Bareli', 'Raebareli'] (Score: 95)
In Uttar Pradesh: Potential duplicates ['Maharajganj', 'Mahrajganj'] (Score: 95)
In Uttar Pradesh: Potential duplicates ['Bulandshahar', 'Bulandshahr'] (Score: 96)
In Uttar Pradesh: Potential duplicates ['Gautam Buddha Nagar', 'Kanpur Nagar'] (Score: 86)
In Uttar Pradesh: Potential duplicates ['Kanpur Nagar', 'Sant Ravidas Nagar'] (Score: 86)
In Uttar Pradesh: Potential duplicates ['Baghpat', 'Baghpat *'] (Score: 100)
In Uttar Pradesh: Potential duplicates ['Kushinagar', 'Kushinagar *

In [15]:
import pandas as pd

# 1. Standardize formatting to handle case and trailing spaces first
df_demo['district'] = df_demo['district'].str.strip()

# 2. Comprehensive Manual Mapping
district_master_map = {
    'Uttar Pradesh': {
        'Bara Banki': 'Barabanki', 'Rae Bareli': 'Raebareli', 'Mahrajganj': 'Maharajganj',
        'Bulandshahar': 'Bulandshahr', 'Bagpat': 'Baghpat', 'Sant Ravidas Nagar Bhadohi': 'Sant Ravidas Nagar',
        'Jyotiba Phule Nagar': 'Amroha', # Amroha is the current official name
        'Kushi Nagar': 'Kushinagar'
    },
    'Andhra Pradesh': {
        'chittoor': 'Chittoor', 'Ananthapuramu': 'Ananthapur', 'Rangareddi': 'Rangareddy',
        'rangareddi': 'Rangareddy', 'Mahbubnagar': 'Mahabubnagar', 'Mahabub Nagar': 'Mahabubnagar',
        'Karim Nagar': 'Karimnagar', 'K.v. Rangareddy': 'Rangareddy', 'K.V.Rangareddy': 'Rangareddy'
    },
    'Karnataka': {
        'Tumkur': 'Tumakuru', 'Hasan': 'Hassan', 'Bijapur(KAR)': 'Vijayapura',
        'Davanagere': 'Davangere', 'Chamrajanagar': 'Chamarajanagar', 'Chamrajnagar': 'Chamarajanagar',
        'Chikmagalur': 'Chikkamagaluru', 'Chickmagalur': 'Chikkamagaluru', 'yadgir': 'Yadgir',
        'Bangalore': 'Bengaluru Urban', 'Bangalore Rural': 'Bengaluru Rural'
    },
    'West Bengal': {
        'HOWRAH': 'Howrah', 'HOOGHLY': 'Hooghly', 'Puruliya': 'Purulia',
        'South 24 parganas': 'South 24 Parganas', 'KOLKATA': 'Kolkata',
        'Barddhaman': 'Bardhaman', 'MALDA': 'Malda', 'Maldah': 'Malda',
        'East Midnapur': 'East Midnapore', 'East midnapore': 'East Midnapore',
        'Hooghiy': 'Hooghly', 'hooghly': 'Hooghly', 'NADIA': 'Nadia',
        'South 24 pargana': 'South 24 Parganas', 'South 24 Pargana': 'South 24 Parganas'
    },
    'Maharashtra': {
        'Ahmed Nagar': 'Ahmednagar', 'Ahmadnagar': 'Ahmednagar', 'Buldana': 'Buldhana',
        'Dist : Thane': 'Thane', 'Raigarh(MH)': 'Raigarh', 'Mumbai City': 'Mumbai',
        'Chatrapati Sambhaji Nagar': 'Chhatrapati Sambhajinagar', 'Gondiya': 'Gondia',
        'Mumbai( Sub Urban )': 'Mumbai Suburban'
    },
    'Telangana': {
        'Warangal Rural': 'Warangal Rural', # Keep distinct
        'Warangal (urban)': 'Warangal Urban',
        'Jangoan': 'Jangaon',
        'Medchal-malkajgiri': 'Medchal-Malkajgiri',
        'Medchal?malkajgiri': 'Medchal-Malkajgiri',
        'Medchal−malkajgiri': 'Medchal-Malkajgiri',
        'Medchalâˆ’malkajgiri': 'Medchal-Malkajgiri'
    },
    'Odisha': {
        'JAJPUR': 'Jajpur', 'jajpur': 'Jajpur', 'Jajapur': 'Jajpur',
        'Baleswar': 'Baleshwar', 'Khorda': 'Khordha', 'ANGUL': 'Angul',
        'NUAPADA': 'Nuapada', 'Jagatsinghapur': 'Jagatsinghpur',
        'Sundergarh': 'Sundargarh', 'ANUGUL': 'Angul', 'Anugul': 'Angul'
    },
    'Bihar': {
        'Sheikpura': 'Sheikhpura', 'Samstipur': 'Samastipur',
        'Kaimur (Bhabua)': 'Kaimur', 'Aurangabad(BH)': 'Aurangabad',
        'Aurangabad(bh)': 'Aurangabad'
    },
    'Punjab': {
        'Sri Muktsar Sahib': 'Muktsar',
        'S.A.S Nagar(Mohali)': 'Sas Nagar (Mohali)',
        'SAS Nagar (Mohali)': 'Sas Nagar (Mohali)'
    },
    'Chhattisgarh': {
        'Uttar Bastar Kanker': 'Kanker',
        'Janjgir-champa': 'Janjgir Champa',
        'Janjgir - Champa': 'Janjgir Champa',
        'Mohla-Manpur-Ambagarh Chouki': 'Mohalla-Manpur-Ambagarh Chowki'
    },
    'Sikkim': {
        'North': 'North Sikkim', 'South': 'South Sikkim', 'West': 'West Sikkim'
    }
}

# 3. Apply the Mapping Surgicaly
for state, mapping in district_master_map.items():
    for wrong, right in mapping.items():
        mask = (df_demo['state'] == state) & (df_demo['district'] == wrong)
        df_demo.loc[mask, 'district'] = right

# 4. Global Regex Cleanup (Handles the * and \xa0 issues across all rows)
df_demo['district'] = df_demo['district'].str.replace(r'[*]', '', regex=True)
df_demo['district'] = df_demo['district'].str.replace(r'\xa0', ' ', regex=True)
df_demo['district'] = df_demo['district'].str.strip()

print("Manual District Cleanup for Demographic Data Complete!")

Manual District Cleanup for Demographic Data Complete!


In [16]:
from fuzzywuzzy import process

# 1. Get list of unique states from your cleaned df_demo
states = df_demo['state'].unique()

print("Final Audit: Searching for remaining District Duplicates...")
print("-" * 60)

found_any = False
for state in states:
    # Filter districts for the current state
    districts = df_demo[df_demo['state'] == state]['district'].unique().tolist()
    
    if len(districts) < 2:
        continue
        
    for dist in districts:
        # Find matches within the same state
        matches = process.extract(dist, [d for d in districts if d != dist], limit=1)
        
        for match, score in matches:
            # We check for a high score to see if any typos survived
            if score >= 88:
                # Filter out the legitimate directional districts we intentionally kept
                directional_keywords = ['East', 'West', 'North', 'South', 'Central']
                is_legit_directional = any(kw in dist for kw in directional_keywords) and any(kw in match for kw in directional_keywords)
                
                if not is_legit_directional:
                    print(f"[{state}] Remaining Duplicate: {dist} <---> {match} (Score: {score})")
                    found_any = True
                
                # Remove to avoid printing the reverse pair
                if match in districts:
                    districts.remove(match)

if not found_any:
    print("Zero typos found! Only unique and legitimate districts remain.")
print("-" * 60)

Final Audit: Searching for remaining District Duplicates...
------------------------------------------------------------
[Uttar Pradesh] Remaining Duplicate: Prayagraj <---> Agra (Score: 90)
[Uttar Pradesh] Remaining Duplicate: Balrampur <---> Rampur (Score: 90)
[Andhra Pradesh] Remaining Duplicate: Ananthapur <---> Anantapur (Score: 95)
[Andhra Pradesh] Remaining Duplicate: Sri Potti Sriramulu Nellore <---> Nellore (Score: 90)
[Gujarat] Remaining Duplicate: Ahmedabad <---> Ahmadabad (Score: 89)
[Gujarat] Remaining Duplicate: Sabarkantha <---> Sabar Kantha (Score: 96)
[Gujarat] Remaining Duplicate: Surendra Nagar <---> Surendranagar (Score: 96)
[Gujarat] Remaining Duplicate: Banaskantha <---> Banas Kantha (Score: 96)
[Gujarat] Remaining Duplicate: Panchmahals <---> Panch Mahals (Score: 96)
[Rajasthan] Remaining Duplicate: Jalor <---> Jalore (Score: 91)
[Rajasthan] Remaining Duplicate: Jhunjhunun <---> Jhunjhunu (Score: 95)
[Karnataka] Remaining Duplicate: Bengaluru <---> Bengaluru Rura

In [17]:
# 1. Final comprehensive mapping for the leftovers
final_surgical_map = {
    'Andhra Pradesh': {
        'Ananthapur': 'Anantapur',
        'Sri Potti Sriramulu Nellore': 'Nellore'
    },
    'Gujarat': {
        'Ahmadabad': 'Ahmedabad',
        'Sabar Kantha': 'Sabarkantha',
        'Surendra Nagar': 'Surendranagar',
        'Banas Kantha': 'Banaskantha',
        'Panch Mahals': 'Panchmahals'
    },
    'Rajasthan': {
        'Jalor': 'Jalore',
        'Jhunjhunun': 'Jhunjhunu'
    },
    'West Bengal': {
        'nadia': 'Nadia',
        'east midnapore': 'East Midnapore'
    },
    'Telangana': {
        'Warangal': 'Warangal Urban', # Mapping the generic 'Warangal' to Urban
        'Medchalâˆ’malkajgiri': 'Medchal-Malkajgiri',
        'Jangoan': 'Jangaon'
    },
    'Odisha': {
        'Bhadrak(R)': 'Bhadrak'
    },
    'Kerala': {
        'Kasargod': 'Kasaragod'
    },
    'Tamil Nadu': {
        'Kanniyakumari': 'Kanyakumari',
        'Thiruvallur': 'Tiruvallur',
        'Viluppuram': 'Villupuram',
        'Tirupathur': 'Tirupattur',
        'Thiruvarur': 'Tiruvarur'
    },
    'Jharkhand': {
        'Palamau': 'Palamu',
        'Sahebganj': 'Sahibganj',
        'Hazaribag': 'Hazaribagh',
        'Seraikela-kharsawan': 'Seraikela-Kharsawan',
        'Pakaur': 'Pakur'
    },
    'Jammu and Kashmir': {
        'Leh (ladakh)': 'Leh'
    },
    'Mizoram': {
        'Mammit': 'Mamit'
    },
    'Himachal Pradesh': {
        'Lahul and Spiti': 'Lahaul and Spiti',
        'Lahul & Spiti': 'Lahaul and Spiti'
    },
    'Haryana': {
        'Yamuna Nagar': 'Yamunanagar'
    },
    'Uttarakhand': {
        'Hardwar': 'Haridwar'
    },
    'Dadra and Nagar Haveli and Daman and Diu': {
        'Dadra & Nagar Haveli': 'Dadra and Nagar Haveli'
    },
    'Andaman and Nicobar Islands': {
        'Nicobars': 'Nicobar'
    }
}

# Apply the final mapping
for state, mapping in final_surgical_map.items():
    for wrong, right in mapping.items():
        mask = (df_demo['state'] == state) & (df_demo['district'] == wrong)
        df_demo.loc[mask, 'district'] = right

# One last global strip to kill invisible spaces
df_demo['district'] = df_demo['district'].str.strip()
df_demo['state'] = df_demo['state'].str.strip()



In [18]:
from fuzzywuzzy import process

# Get unique states from the now-cleaned df_demo
states = df_demo['state'].unique()

print("FINAL AUDIT: Scanning for any surviving issues...")
print("-" * 60)

survivors_found = False
for state in states:
    districts = df_demo[df_demo['state'] == state]['district'].unique().tolist()
    
    if len(districts) < 2:
        continue
        
    for dist in districts:
        matches = process.extract(dist, [d for d in districts if d != dist], limit=1)
        
        for match, score in matches:
            if score >= 88:
                # Filter out the legitimate directional districts
                directional_keywords = ['East', 'West', 'North', 'South', 'Central', 'Urban', 'Rural']
                is_legit = any(kw in dist for kw in directional_keywords) and any(kw in match for kw in directional_keywords)
                
                if not is_legit:
                    print(f"[{state}] Remaining Duplicate: {dist} <---> {match} (Score: {score})")
                    survivors_found = True
                
                if match in districts:
                    districts.remove(match)

if not survivors_found:
    print("Verification Complete: No accidental duplicates detected.")
else:
    print("Some survivors remain. See list above.")
print("-" * 60)

FINAL AUDIT: Scanning for any surviving issues...
------------------------------------------------------------
[Uttar Pradesh] Remaining Duplicate: Prayagraj <---> Agra (Score: 90)
[Uttar Pradesh] Remaining Duplicate: Balrampur <---> Rampur (Score: 90)
[Karnataka] Remaining Duplicate: Bengaluru <---> Bengaluru Rural (Score: 90)
[Karnataka] Remaining Duplicate: Bengaluru Urban <---> Bengaluru (Score: 90)
[West Bengal] Remaining Duplicate: Paschim Medinipur <---> Medinipur (Score: 90)
[West Bengal] Remaining Duplicate: Paschim Bardhaman <---> Bardhaman (Score: 90)
[Telangana] Remaining Duplicate: Sangareddy <---> Rangareddy (Score: 90)
[Telangana] Remaining Duplicate: Medchal-Malkajgiri <---> Medchalâmalkajgiri (Score: 97)
[Odisha] Remaining Duplicate: Jajpur <---> Jajapur (Score: 92)
[Maharashtra] Remaining Duplicate: Mumbai <---> Mumbai Suburban (Score: 90)
[Maharashtra] Remaining Duplicate: Gondia <---> Gondiya (Score: 92)
[Assam] Remaining Duplicate: Karbi Anglong <---> West Karbi 

In [19]:
import re

# 1. Targeted regex to replace anything between 'Medchal' and 'malkajgiri' with a standard hyphen
df_demo['district'] = df_demo['district'].str.replace(r'Medchal.*malkajgiri', 'Medchal-Malkajgiri', case=False, regex=True)

# 2. Force capitalization to ensure 'Medchal-Malkajgiri' is the only version
mask = df_demo['district'].str.contains('Medchal', case=False, na=False)
df_demo.loc[mask, 'district'] = 'Medchal-Malkajgiri'

# 3. Double check for any other hidden special characters in the whole column
# This removes characters that aren't alphanumeric, spaces, or standard hyphens
df_demo['district'] = df_demo['district'].apply(lambda x: re.sub(r'[^\w\s-]', '', str(x)))

print("Medchal-Malkajgiri encoding error fixed.")

Medchal-Malkajgiri encoding error fixed.


In [20]:
# Group by state and district to keep unique districts (like Aurangabad) separate
district_totals = df_demo.groupby(['state', 'district'])['total_updates'].sum()

# Sort by the highest updates first and print the full list
print("--- CLEANED DISTRICT-WISE TOTALS ---")
print(district_totals.sort_values(ascending=False).to_string())

--- CLEANED DISTRICT-WISE TOTALS ---
state                                     district                      
Maharashtra                               Thane                             447254
                                          Pune                              438478
West Bengal                               South 24 Parganas                 401229
                                          Murshidabad                       371953
Gujarat                                   Surat                             357582
Delhi                                     North West Delhi                  313989
Karnataka                                 Bengaluru                         303924
West Bengal                               North 24 Parganas                 290477
Gujarat                                   Ahmedabad                         275644
Rajasthan                                 Jaipur                            275340
West Bengal                               Uttar Dinajpur    

In [21]:
# 1. Direct fix for the landmark/address junk
landmark_to_district = {
    'West Bengal': {
        'South DumDumM': 'North 24 Parganas',
        'South  Twenty Four Parganas': 'South 24 Parganas',
        'North Twenty Four Parganas': 'North 24 Parganas',
        'South Twenty Four Parganas': 'South 24 Parganas'
    },
    'Bihar': {
        'Near University Thana': 'Darbhanga' # University Thana is in Darbhanga
    },
    'Rajasthan': {
        'Near meera hospital': 'Sriganganagar' # Meera Hospital is a major landmark there
    },
    'Karnataka': {
        '5th cross': 'Bengaluru Urban'
    },
    'Tamil Nadu': {
        'Near Dhyana Ashram': 'Chennai'
    },
    'Maharashtra': {
        'Near Uday nagar NIT garden': 'Nagpur'
    }
}

# Apply the mapping
for state, mapping in landmark_to_district.items():
    for junk, correct in mapping.items():
        mask = (df_demo['state'] == state) & (df_demo['district'] == junk)
        df_demo.loc[mask, 'district'] = correct

# 2. Fix the naming style for the new Chhattisgarh district
df_demo.loc[df_demo['district'] == 'ManendragarhChirmiriBharatpur', 'district'] = 'Manendragarh-Chirmiri-Bharatpur'

# 3. Final normalization to catch any double spaces or case issues
df_demo['district'] = df_demo['district'].str.replace(r'\s+', ' ', regex=True).str.strip()

print("Landmarks and spacing issues corrected.")

Landmarks and spacing issues corrected.


In [22]:
# Group by state and district to keep unique districts (like Aurangabad) separate
district_totals = df_demo.groupby(['state', 'district'])['total_updates'].sum()

# Sort by the highest updates first and print the full list
print("--- CLEANED DISTRICT-WISE TOTALS ---")
print(district_totals.sort_values(ascending=False).to_string())

--- CLEANED DISTRICT-WISE TOTALS ---
state                                     district                       
Maharashtra                               Thane                              447254
West Bengal                               South 24 Parganas                  443300
Maharashtra                               Pune                               438478
West Bengal                               Murshidabad                        371953
Gujarat                                   Surat                              357582
Delhi                                     North West Delhi                   313989
West Bengal                               North 24 Parganas                  310220
Karnataka                                 Bengaluru                          303924
Gujarat                                   Ahmedabad                          275644
Rajasthan                                 Jaipur                             275340
West Bengal                               Uttar D

In [23]:
# Mapping KV Rangareddy to your standardized Rangareddy name
mask = (df_demo['state'] == 'Telangana') & (df_demo['district'] == 'Kv Rangareddy')
df_demo.loc[mask, 'district'] = 'Rangareddy'

# Also check for any weird spacing or case variations again
df_demo['district'] = df_demo['district'].str.replace('K.V. Rangareddy', 'Rangareddy', case=False)
df_demo['district'] = df_demo['district'].str.replace('K.V.Rangareddy', 'Rangareddy', case=False)

print("KV Rangareddy unified with Rangareddy.")

KV Rangareddy unified with Rangareddy.


In [24]:
# Group by state and district to keep unique districts (like Aurangabad) separate
district_totals = df_demo.groupby(['state', 'district'])['total_updates'].sum()

# Sort by the highest updates first and print the full list
print("--- CLEANED DISTRICT-WISE TOTALS ---")
print(district_totals.sort_values(ascending=False).to_string())

--- CLEANED DISTRICT-WISE TOTALS ---
state                                     district                       
Maharashtra                               Thane                              447254
West Bengal                               South 24 Parganas                  443300
Maharashtra                               Pune                               438478
West Bengal                               Murshidabad                        371953
Gujarat                                   Surat                              357582
Delhi                                     North West Delhi                   313989
West Bengal                               North 24 Parganas                  310220
Karnataka                                 Bengaluru                          303924
Gujarat                                   Ahmedabad                          275644
Rajasthan                                 Jaipur                             275340
West Bengal                               Uttar D

In [25]:
# Group by state and sum the updates
state_totals = df_demo.groupby('state')['total_updates'].sum().sort_values(ascending=False)

# Print the full list to ensure no 'ghost' states remain at the bottom
print("--- FINAL STATE-WISE DEMOGRAPHIC TOTALS ---")
print(state_totals.to_string())

--- FINAL STATE-WISE DEMOGRAPHIC TOTALS ---
state
Uttar Pradesh                               8542328
Maharashtra                                 5054603
Bihar                                       4814352
West Bengal                                 3872737
Madhya Pradesh                              2912938
Rajasthan                                   2817617
Andhra Pradesh                              2295584
Tamil Nadu                                  2212229
Chhattisgarh                                2005438
Gujarat                                     1824327
Karnataka                                   1695286
Telangana                                   1629910
Delhi                                       1438934
Jharkhand                                   1401189
Haryana                                     1166140
Odisha                                      1112065
Assam                                       1012578
Punjab                                       881895
Kerala        

In [26]:

df_demo.to_csv('Cleaned Datasets/cleaned_demographic.csv', index=False)

print("File 'cleaned_demographic.csv' has been saved successfully!")

File 'cleaned_demographic.csv' has been saved successfully!


In [27]:
df_demo.head(20)

,date,state,district,pincode,demo_age_5_17,demo_age_17_,total_updates
0,2025-03-01,Uttar Pradesh,Gorakhpur,273213,49,529,578
1,2025-03-01,Andhra Pradesh,Chittoor,517132,22,375,397
2,2025-03-01,Gujarat,Rajkot,360006,65,765,830
3,2025-03-01,Andhra Pradesh,Srikakulam,532484,24,314,338
4,2025-03-01,Rajasthan,Udaipur,313801,45,785,830
5,2025-03-01,Rajasthan,Sikar,332028,28,285,313
6,2025-03-01,Karnataka,Tumakuru,572201,88,332,420
7,2025-03-01,Uttar Pradesh,Gorakhpur,273211,61,836,897
8,2025-03-01,Andhra Pradesh,Kurnool,518313,83,986,1069
9,2025-03-01,West Bengal,Paschim Medinipur,721148,13,281,294
